In [ ]:
import os
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor, as_completed
from modules.StrategySearcher import StrategySearcher
import warnings
warnings.filterwarnings("ignore")

label_methods = [
    "atr",
    # "trend",
    # "trend_profit",
    # "trend_multi",
    # "clusters",
    # "multi_window",
    # "validated_levels",
    # "zigzag",
    # "mean_rev",
    # "mean_rev_multi",
    # "mean_rev_vol",
    # "filter",
    # "multi_filter",
    # "filter_bidirectional",
    # "filter_one",
    # "trend_one",
    # "filter_flat",
]

base_cfg = dict(
    symbol='XAUUSD', timeframe='H1', direction='both',
    train_start=datetime(2019,1,1), train_end=datetime(2025,1,1),
    test_start=datetime(2022,6,1),  test_end=datetime(2023,6,1),
    search_type='clusters', n_trials=500, n_models=1, debug=True
)

configs = []
for lm in label_methods:
    cfg = base_cfg.copy()
    cfg["label_method"] = lm
    configs.append(cfg)

# Calcular núcleos por configuración
# total_cores = os.cpu_count()
# cores_per_config = max(1, total_cores // len(configs))

# Actualizar n_jobs en cada configuración
for cfg in configs:
    #cfg["n_jobs"] = cores_per_config
    cfg["tag"] = f"{cfg['symbol']} {cfg['timeframe']} {cfg['direction']} {cfg['label_method']} {cfg['search_type']} {cfg.get('search_subtype', '')}".strip()

def launch(cfg):
    s = StrategySearcher(**cfg)
    s.run_search()

with ProcessPoolExecutor(max_workers=len(configs)) as pool:
    futures = {pool.submit(launch, c): c["tag"] for c in configs}
    for f in as_completed(futures):
        tag = futures[f]
        try:
            print(f"[{tag}] terminado")
        except Exception as e:
            print(f"[{tag}] falló: {e}")

🔍 DEBUG: base_df.shape = (42333, 5)
🔍 DEBUG: train_start = 2019-01-01 00:00:00, train_end = 2025-01-01 00:00:00
🔍 DEBUG: test_start = 2022-06-01 00:00:00, test_end = 2023-06-01 00:00:00
🔍 DEBUG: pad = 199
🔍 DEBUG: bar_delta = 0 days 01:00:00
🔍 DEBUG: start_ext = 2018-12-23 17:00:00, end_ext = 2025-01-01 00:00:00
🔍 DEBUG: ds_slice.shape = (35604, 5)
🔍 DEBUG: full_ds.shape después de get_features = (35405, 12)
🔍 DEBUG: full_ds.shape después de apply_labeling = (17446, 13)
🔍 DEBUG: full_ds.shape después de recorte = (17446, 13)
🔍 DEBUG: Columnas problemáticas eliminadas: 1
🔍 DEBUG: test_mask.sum() = 2971
🔍 DEBUG: train_mask.sum() = 17446
🔍 DEBUG: train_mask.sum() después de evitar solapamiento = 14475
🔍 DEBUG: train_data.shape final = (14475, 12)
🔍 DEBUG: test_data.shape final = (2971, 12)
🔍 DEBUG evaluate_clusters: ds_train.shape = (14475, 13)
🔍 DEBUG evaluate_clusters: ds_test.shape = (2971, 12)
🔍 DEBUG evaluate_clusters: cluster_sizes = labels_meta
 3    5499
 4    3715
 1    3309
 5  